<h1>Model Building-1 </h1>

In this notebook we prepare the dataset we flattened and run multiple models with a differnt number of features. Each model we test also goes through loops for hyperparamter tuning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from datetime import datetime
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib
from sklearn import tree

In [2]:
df = pd.read_csv('punt_df2.csv')

In [3]:
df = df.drop(columns=['Unnamed: 0','kickoffReturnFormation','returnDirectionIntended','kickDirectionActual.1','down',
                                 'penaltyCodes','penaltyYards'])

In [4]:
df

,gameId,playId,snapDetail,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,kickContactType,quarter,...,P22_t25_dis,P22_t25_o,P22_t25_dir,P22_t30_x,P22_t30_y,P22_t30_s,P22_t30_a,P22_t30_dis,P22_t30_o,P22_t30_dir
0,2018090600,366,OK,2.12,4.46,N,C,C,CC,1,...,0.29,234.56,239.04,13.87,32.05,1.26,5.02,0.16,232.79,208.81
1,2018090600,872,OK,2.00,4.35,N,C,L,BF,1,...,0.36,261.29,298.83,41.28,32.59,3.61,2.63,0.37,271.19,321.46
2,2018090600,973,OK,2.19,4.98,N,L,L,CC,2,...,0.39,137.01,261.71,14.11,27.14,3.19,2.72,0.32,142.72,244.21
3,2018090600,1850,OK,2.00,4.53,N,R,R,BB,2,...,0.30,81.98,71.33,83.63,30.32,1.51,1.80,0.16,63.54,75.25
4,2018090600,1989,OK,2.12,4.62,N,C,C,CC,2,...,0.17,87.27,165.12,33.93,20.29,2.85,3.29,0.27,91.30,109.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5361,2021010315,1359,H,2.23,4.50,N,L,L,CC,2,...,0.73,239.92,264.21,81.01,42.26,8.02,2.08,0.80,230.31,255.97
5362,2021010315,1479,OK,2.07,4.80,N,R,R,CC,2,...,0.51,196.28,153.07,36.96,40.09,7.03,2.89,0.69,153.27,143.08
5363,2021010315,2235,OK,2.08,3.36,N,C,C,CC,3,...,0.62,184.74,159.87,39.97,37.73,7.70,2.22,0.76,161.90,150.72
5364,2021010315,2497,OK,1.99,4.69,N,C,C,CC,3,...,0.51,198.05,152.45,39.46,39.41,6.62,1.73,0.65,176.24,146.87


Doing some data exploration looking at the spread of our data. We are looking at how many punts resulted in fair catch or returns. Verifying that our data has 5 as a possible value for the quarter column. Concluded that the 5th quarter meant overtime.

In [5]:
df["specialTeamsResult"].value_counts()/5366
#df.loc[df['specialTeamsResult'] == "Fair Catch"]

Return                      0.426016
Fair Catch                  0.305628
Downed                      0.154491
Touchback                   0.071375
Muffed                      0.028699
Non-Special Teams Result    0.013791
Name: specialTeamsResult, dtype: float64

In [7]:
df["specialTeamsResult"].unique()
df.loc[df['specialTeamsResult'] == "Fair Catch"]

,gameId,playId,snapDetail,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,kickContactType,quarter,...,P22_t25_dis,P22_t25_o,P22_t25_dir,P22_t30_x,P22_t30_y,P22_t30_s,P22_t30_a,P22_t30_dis,P22_t30_o,P22_t30_dir
2,2018090600,973,OK,2.19,4.98,N,L,L,CC,2,...,0.39,137.01,261.71,14.11,27.14,3.19,2.72,0.32,142.72,244.21
5,2018090600,2406,OK,2.04,4.69,N,C,C,CC,3,...,0.23,74.34,77.99,72.39,30.19,1.62,1.24,0.17,67.37,68.41
21,2018090900,4325,OK,2.08,3.80,A,R,R,CC,4,...,0.13,78.48,114.83,65.90,26.45,1.50,1.36,0.15,70.08,166.53
29,2018090901,2907,OK,2.14,4.22,A,L,L,CC,3,...,0.17,252.94,128.13,61.73,20.15,2.16,1.57,0.22,263.78,155.38
30,2018090901,3215,OK,2.28,4.08,A,L,L,CC,4,...,0.11,69.24,55.82,47.50,22.63,1.11,2.07,0.10,69.64,301.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352,2021010314,1771,OK,2.20,4.67,N,C,C,CC,2,...,0.68,95.85,91.49,47.56,44.17,7.75,1.99,0.77,107.72,99.38
5357,2021010315,283,OK,2.03,4.25,A,L,L,CC,1,...,0.44,216.03,265.81,38.61,34.51,1.69,5.90,0.19,213.37,301.41
5358,2021010315,386,OK,2.00,4.76,N,R,R,CC,1,...,0.54,343.71,329.98,79.69,16.23,7.16,3.47,0.71,317.47,310.85
5359,2021010315,553,H,2.10,4.63,N,L,L,CC,1,...,0.37,34.54,66.41,40.48,11.37,4.87,1.73,0.48,48.96,78.94


In [ ]:
df.loc[df['specialTeamsResult'] == "Return"]

In [9]:
print(df['hangTime'].mean())

4.353382519863793


one hot encoding for our categorical values. Here we create dummies for every field that needs to be encoded into binary columns. Then we concatenate the dummies into the dataframe now called df1 then we drop the original columns that contained the information

In [10]:
dfsnap =pd.get_dummies(df["snapDetail"],prefix="snap")
dfkt =pd.get_dummies(df["kickType"],prefix="kicktype")
dfdi =pd.get_dummies(df["kickDirectionIntended"], prefix="kickDirintended")
dfka= pd.get_dummies(df["kickDirectionActual"],prefix="kickDirActual")
dfqu=pd.get_dummies(df["quarter"], prefix="quarter")
dfstr=pd.get_dummies(df['specialTeamsResult'],prefix="result")
dfpd=pd.get_dummies(df['playDirection'],prefix="playDirection")

In [11]:
df1= pd.concat([dfsnap,dfkt,dfdi,dfka,dfqu,dfstr,dfpd,df], axis=1) #dropped kickcontacttpe bc dataleakage
df1= df1.drop(columns=['snapDetail', 'kickType','kickDirectionIntended','kickDirectionActual','kickContactType','quarter','specialTeamsResult','playDirection','playResult'])
df1
#array of column names


,snap_<,snap_>,snap_H,snap_L,snap_OK,kicktype_A,kicktype_N,kicktype_R,kickDirintended_C,kickDirintended_L,...,P22_t25_dis,P22_t25_o,P22_t25_dir,P22_t30_x,P22_t30_y,P22_t30_s,P22_t30_a,P22_t30_dis,P22_t30_o,P22_t30_dir
0,0,0,0,0,1,0,1,0,1,0,...,0.29,234.56,239.04,13.87,32.05,1.26,5.02,0.16,232.79,208.81
1,0,0,0,0,1,0,1,0,1,0,...,0.36,261.29,298.83,41.28,32.59,3.61,2.63,0.37,271.19,321.46
2,0,0,0,0,1,0,1,0,0,1,...,0.39,137.01,261.71,14.11,27.14,3.19,2.72,0.32,142.72,244.21
3,0,0,0,0,1,0,1,0,0,0,...,0.30,81.98,71.33,83.63,30.32,1.51,1.80,0.16,63.54,75.25
4,0,0,0,0,1,0,1,0,1,0,...,0.17,87.27,165.12,33.93,20.29,2.85,3.29,0.27,91.30,109.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5361,0,0,1,0,0,0,1,0,0,1,...,0.73,239.92,264.21,81.01,42.26,8.02,2.08,0.80,230.31,255.97
5362,0,0,0,0,1,0,1,0,0,0,...,0.51,196.28,153.07,36.96,40.09,7.03,2.89,0.69,153.27,143.08
5363,0,0,0,0,1,0,1,0,1,0,...,0.62,184.74,159.87,39.97,37.73,7.70,2.22,0.76,161.90,150.72
5364,0,0,0,0,1,0,1,0,1,0,...,0.51,198.05,152.45,39.46,39.41,6.62,1.73,0.65,176.24,146.87


Continuing to format the data to be compatible with model building. Getting rid of string values with binary values and reformatting time columns. Also dropping more columns that do not seem to be necessary for the model.Moving fair catch column to the last position in order to make it easier to select. 

In [12]:
df1 = df1.drop(columns = ['possessionTeam','homeTeamAbbr','visitorTeamAbbr','yardlineSide'])

In [13]:
df1['gameClock']=pd.to_datetime(df1['gameClock']).astype('int64')
# df1['gameClock'][:5]

In [14]:
df1[df1.columns[df1.isnull().sum() > 0]].isnull().sum()

operationTime          75
hangTime               80
kickLength             74
kickReturnYardage    3011
dtype: int64

In [15]:
df1.drop(columns=['operationTime', 'hangTime', 'kickLength', 'kickReturnYardage'], inplace=True)

In [16]:
df1.drop(columns=['result_Downed', 'result_Muffed', 
                  'result_Non-Special Teams Result', 'result_Return', 'result_Touchback'],
                 inplace=True)
         

In [17]:
# moves fair catch prediction column to the end of the dataframe
df1 = df1[[c for c in df1 if c not in ['result_Fair Catch']] 
       + ['result_Fair Catch']]

Functions to help with analysis and clearly seeing columns with index

In [18]:
def print_analysis(y_pred,y_v):
    cnf_matrix = metrics.confusion_matrix(y_v, y_pred)
    #ax= plt.subplot()
    #sns.heatmap(cnf_matrix, annot=True, fmt='g', ax=ax, cmap='Blues');  #annot=True to annotate cells, ftm='g' to disable scientific notation# labels, title and ticks
    #ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    #ax.set_title('Confusion Matrix'); 
    #ax.xaxis.set_ticklabels(['Not fair catch', 'fair catch']); ax.yaxis.set_ticklabels(['Not fair catch', ' fair catch']);
    print("Confusion matrix: ")
    print(cnf_matrix)
    print("")
    print("Accuracy:",round(metrics.accuracy_score(y_v, y_pred),4))
    print("Precision:",round(metrics.precision_score(y_v, y_pred, zero_division=1),4))
    print("Recall:",round(metrics.recall_score(y_v, y_pred),4))

    auc = metrics.roc_auc_score(y_v, y_pred)
    print("AUC:",round(auc,4))
    print("")
    return
def calculation(model_name,x,y,xv,yv):
    model = model_name
    model_name.fit(x,y)
    y_pred = model_name.predict(xv)
    print_analysis(y_pred,yv)
    return
def printCols(data):
    n=0
    for i in data.columns:
        print(i,n)
        n=n+1

Making sure there are no duplicated columns before proceeding to the model building 

In [19]:
df1 = df1.loc[:,~df1.columns.duplicated()]
printCols(df1)

snap_< 0
snap_> 1
snap_H 2
snap_L 3
snap_OK 4
kicktype_A 5
kicktype_N 6
kicktype_R 7
kickDirintended_C 8
kickDirintended_L 9
kickDirintended_R 10
kickDirActual_C 11
kickDirActual_L 12
kickDirActual_R 13
quarter_1 14
quarter_2 15
quarter_3 16
quarter_4 17
quarter_5 18
playDirection_left 19
playDirection_right 20
gameId 21
playId 22
yardsToGo 23
yardlineNumber 24
gameClock 25
preSnapHomeScore 26
preSnapVisitorScore 27
absoluteYardlineNumber 28
nflId 29
puntPlay 30
P1_t0_x 31
P1_t0_y 32
P1_t0_s 33
P1_t0_a 34
P1_t0_dis 35
P1_t0_o 36
P1_t0_dir 37
P1_t5_x 38
P1_t5_y 39
P1_t5_s 40
P1_t5_a 41
P1_t5_dis 42
P1_t5_o 43
P1_t5_dir 44
P1_t10_x 45
P1_t10_y 46
P1_t10_s 47
P1_t10_a 48
P1_t10_dis 49
P1_t10_o 50
P1_t10_dir 51
P1_t15_x 52
P1_t15_y 53
P1_t15_s 54
P1_t15_a 55
P1_t15_dis 56
P1_t15_o 57
P1_t15_dir 58
P1_t20_x 59
P1_t20_y 60
P1_t20_s 61
P1_t20_a 62
P1_t20_dis 63
P1_t20_o 64
P1_t20_dir 65
P1_t25_x 66
P1_t25_y 67
P1_t25_s 68
P1_t25_a 69
P1_t25_dis 70
P1_t25_o 71
P1_t25_dir 72
P1_t30_x 73
P1_t30_

P19_t10_y 928
P19_t10_s 929
P19_t10_a 930
P19_t10_dis 931
P19_t10_o 932
P19_t10_dir 933
P19_t15_x 934
P19_t15_y 935
P19_t15_s 936
P19_t15_a 937
P19_t15_dis 938
P19_t15_o 939
P19_t15_dir 940
P19_t20_x 941
P19_t20_y 942
P19_t20_s 943
P19_t20_a 944
P19_t20_dis 945
P19_t20_o 946
P19_t20_dir 947
P19_t25_x 948
P19_t25_y 949
P19_t25_s 950
P19_t25_a 951
P19_t25_dis 952
P19_t25_o 953
P19_t25_dir 954
P19_t30_x 955
P19_t30_y 956
P19_t30_s 957
P19_t30_a 958
P19_t30_dis 959
P19_t30_o 960
P19_t30_dir 961
P20_t0_x 962
P20_t0_y 963
P20_t0_s 964
P20_t0_a 965
P20_t0_dis 966
P20_t0_o 967
P20_t0_dir 968
P20_t5_x 969
P20_t5_y 970
P20_t5_s 971
P20_t5_a 972
P20_t5_dis 973
P20_t5_o 974
P20_t5_dir 975
P20_t10_x 976
P20_t10_y 977
P20_t10_s 978
P20_t10_a 979
P20_t10_dis 980
P20_t10_o 981
P20_t10_dir 982
P20_t15_x 983
P20_t15_y 984
P20_t15_s 985
P20_t15_a 986
P20_t15_dis 987
P20_t15_o 988
P20_t15_dir 989
P20_t20_x 990
P20_t20_y 991
P20_t20_s 992
P20_t20_a 993
P20_t20_dis 994
P20_t20_o 995
P20_t20_dir 996
P20_t25_

<h2>All information 3 second model</h2>

Scaling the data using MinMax scaler to prepare for model

In [20]:
# need to encode the yardlineSide column by creating a new column 
# to keep track of whether punt is on offensive side or defensive side
X = df1[df1.columns[0:-1]]
y = df1[['result_Fair Catch']]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=0)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
# printTestTrain(X_train, X_test, y_train, y_test)


**logistic Regression**

This section creates a Logistic Regression model and tuning with grid search. The model has the additional play information and all 3 seconds worth of tracking data. 

In [21]:
model = LogisticRegression(random_state=0, max_iter=1000)

In [22]:
model.fit(X_train_scaled, y_train.values.ravel())

LogisticRegression(max_iter=1000, random_state=0)

In [23]:
model.predict_proba(X_test) 

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [24]:
calculation(LogisticRegression(max_iter=1000),X_train_scaled,y_train.values.ravel(),X_test_scaled,y_test)

Confusion matrix: 
[[642 107]
 [251  74]]

Accuracy: 0.6667
Precision: 0.4088
Recall: 0.2277
AUC: 0.5424



In [25]:
for c in [1,2,3,4,5,6,7,8,9,10]: 
    calculation(LogisticRegression(C=c,max_iter=10000),X_train_scaled,y_train.values.ravel(),X_test_scaled,y_test)

Confusion matrix: 
[[642 107]
 [251  74]]

Accuracy: 0.6667
Precision: 0.4088
Recall: 0.2277
AUC: 0.5424

Confusion matrix: 
[[626 123]
 [246  79]]

Accuracy: 0.6564
Precision: 0.3911
Recall: 0.2431
AUC: 0.5394

Confusion matrix: 
[[620 129]
 [245  80]]

Accuracy: 0.6518
Precision: 0.3828
Recall: 0.2462
AUC: 0.537

Confusion matrix: 
[[612 137]
 [244  81]]

Accuracy: 0.6453
Precision: 0.3716
Recall: 0.2492
AUC: 0.5332

Confusion matrix: 
[[611 138]
 [244  81]]

Accuracy: 0.6443
Precision: 0.3699
Recall: 0.2492
AUC: 0.5325

Confusion matrix: 
[[610 139]
 [242  83]]

Accuracy: 0.6453
Precision: 0.3739
Recall: 0.2554
AUC: 0.5349

Confusion matrix: 
[[610 139]
 [241  84]]

Accuracy: 0.6462
Precision: 0.3767
Recall: 0.2585
AUC: 0.5364

Confusion matrix: 
[[608 141]
 [240  85]]

Accuracy: 0.6453
Precision: 0.3761
Recall: 0.2615
AUC: 0.5366

Confusion matrix: 
[[606 143]
 [239  86]]

Accuracy: 0.6443
Precision: 0.3755
Recall: 0.2646
AUC: 0.5368

Confusion matrix: 
[[606 143]
 [239  86]]

Accu

**SVM**

This section creates a Support Vector Machine model and tuning with grid search. The model has the additional play information and all 3 seconds worth of tracking data. 

In [26]:
for c in range(1, 21,2):
    c = c / 10
    print("C: ", c)
    calculation(SVC(C = c),X_train_scaled,y_train.values.ravel(),X_test_scaled,y_test)

C:  0.1
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  0.3
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  0.5
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  0.7
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  0.9
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  1.1
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  1.3
Confusion matrix: 
[[749   0]
 [325   0]]

Accuracy: 0.6974
Precision: 1.0
Recall: 0.0
AUC: 0.5

C:  1.5
Confusion matrix: 
[[748   1]
 [323   2]]

Accuracy: 0.6983
Precision: 0.6667
Recall: 0.0062
AUC: 0.5024

C:  1.7
Confusion matrix: 
[[745   4]
 [319   6]]

Accuracy: 0.6993
Precision: 0.6
Recall: 0.0185
AUC: 0.5066

C:  1.9
Confusion matrix: 
[[736  13]
 [

<b> K-Nearest Neighbors </b> 

For k-nearest neighbors we will attempt to optimize the distance metric and the number of neighbors we are analyzing. This section creates a KNN and tuning with grid search. The model has the additional play information and all 3 seconds worth of tracking data.

Best KNN 3 second model <br>
Distance metric:  Manhattan  
k:  10  
Confusion matrix:   
[[682  67]  
 [275  50]]  

Accuracy: 0.6816  
Precision: 0.4274  
Recall: 0.1538  
AUC: 0.5322  

In [27]:
# Default is p = 2, k = 5
for p in [1, 2]: 
    for k in [1,2,3,4,5,6,7,8,9,10]: 
        distance = {1: "Manhattan", 2: "Euclidean"}
        print("Distance metric: ", distance[p])
        print("k: ", k)
        calculation(KNeighborsClassifier(p=p,n_neighbors=k),X_train_scaled,y_train.values.ravel(),X_test_scaled,y_test)

Distance metric:  Manhattan
k:  1
Confusion matrix: 
[[529 220]
 [219 106]]

Accuracy: 0.5912
Precision: 0.3252
Recall: 0.3262
AUC: 0.5162

Distance metric:  Manhattan
k:  2
Confusion matrix: 
[[674  75]
 [286  39]]

Accuracy: 0.6639
Precision: 0.3421
Recall: 0.12
AUC: 0.5099

Distance metric:  Manhattan
k:  3
Confusion matrix: 
[[568 181]
 [223 102]]

Accuracy: 0.6238
Precision: 0.3604
Recall: 0.3138
AUC: 0.5361

Distance metric:  Manhattan
k:  4
Confusion matrix: 
[[671  78]
 [269  56]]

Accuracy: 0.6769
Precision: 0.4179
Recall: 0.1723
AUC: 0.5341

Distance metric:  Manhattan
k:  5
Confusion matrix: 
[[606 143]
 [234  91]]

Accuracy: 0.649
Precision: 0.3889
Recall: 0.28
AUC: 0.5445

Distance metric:  Manhattan
k:  6
Confusion matrix: 
[[673  76]
 [275  50]]

Accuracy: 0.6732
Precision: 0.3968
Recall: 0.1538
AUC: 0.5262

Distance metric:  Manhattan
k:  7
Confusion matrix: 
[[615 134]
 [243  82]]

Accuracy: 0.649
Precision: 0.3796
Recall: 0.2523
AUC: 0.5367

Distance metric:  Manhatta

**Random Forest Classifier**

For the random forest classifier, we analyzed the size of the forest (number of trees) hyperparameter and used all tracking data with extra play information features. 


Best RFC- 3 second  
Size:  120  
Confusion matrix:   
    [[725  24]  
     [298  27]]  

Accuracy: 0.7002  
Precision: 0.5294  
Recall: 0.0831  
AUC: 0.5255  

In [28]:
for size in range(50,160,10): 
    print("Size: ", size)
    calculation(RandomForestClassifier(n_estimators=size,random_state = 1),X_train_scaled,y_train.values.ravel(),X_test_scaled,y_test)

Size:  50
Confusion matrix: 
[[710  39]
 [297  28]]

Accuracy: 0.6872
Precision: 0.4179
Recall: 0.0862
AUC: 0.517

Size:  60
Confusion matrix: 
[[711  38]
 [296  29]]

Accuracy: 0.689
Precision: 0.4328
Recall: 0.0892
AUC: 0.5192

Size:  70
Confusion matrix: 
[[714  35]
 [295  30]]

Accuracy: 0.6927
Precision: 0.4615
Recall: 0.0923
AUC: 0.5228

Size:  80
Confusion matrix: 
[[714  35]
 [298  27]]

Accuracy: 0.6899
Precision: 0.4355
Recall: 0.0831
AUC: 0.5182

Size:  90
Confusion matrix: 
[[716  33]
 [297  28]]

Accuracy: 0.6927
Precision: 0.459
Recall: 0.0862
AUC: 0.521

Size:  100
Confusion matrix: 
[[718  31]
 [296  29]]

Accuracy: 0.6955
Precision: 0.4833
Recall: 0.0892
AUC: 0.5239

Size:  110
Confusion matrix: 
[[720  29]
 [299  26]]

Accuracy: 0.6946
Precision: 0.4727
Recall: 0.08
AUC: 0.5206

Size:  120
Confusion matrix: 
[[725  24]
 [298  27]]

Accuracy: 0.7002
Precision: 0.5294
Recall: 0.0831
AUC: 0.5255

Size:  130
Confusion matrix: 
[[722  27]
 [301  24]]

Accuracy: 0.6946
Prec

In [ ]:
RFC=RandomForestClassifier(n_estimators=120,random_state=1).fit(X_train_scaled,y_train.values.ravel())
RFC.fit(X_train_scaled,y_train.values.ravel())
y_pred = RFC.predict(X_test_scaled)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
ax= plt.subplot()
sns.heatmap(cnf_matrix, annot=True, fmt='g', ax=ax, cmap='Blues');  #annot=True to annotate cells, ftm='g' to disable scientific notation# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Not fair catch', 'fair catch']); ax.yaxis.set_ticklabels(['Not fair catch', ' fair catch']);
print("Confusion matrix: ")
print(cnf_matrix)

**Decison Tree Classifier** 

Creating a Decision Tree model and changing the criterion and splitter paramaters for grid search. Model uses all 3 seconds and extra play information. 

Found these to be the best:  
    
    Criterion:  entropy    
    Splitter:  best    
    Confusion matrix:     
        [[539 210]    
         [206 119]]      
    Accuracy: 0.6127    
    Precision: 0.3617    
    Recall: 0.3662    
    AUC: 0.5429     

    max_depth=3       
    Confusion matrix:     
        [[733  16]    
        [313  12]]    
    Accuracy: 0.6937    
    Precision: 0.4286    
    Recall: 0.0369    
    AUC: 0.5078    

In [ ]:
for criterion in ["gini","entropy"]:
    for splitter in ["best","random"]:
            print("Criterion: ", criterion)
            print("Splitter: ", splitter)
            calculation(DecisionTreeClassifier(criterion=criterion, splitter=splitter),X_train_scaled, y_train.values.ravel(),X_test_scaled,y_test)

In [ ]:
from sklearn import tree
dt3 = DecisionTreeClassifier(criterion='entropy', splitter= 'best', max_depth=3)
dt3= dt3.fit(X_train_scaled, y_train)
#y_pred = model_name.predict(X_test)
#visualizing gini-best tree
fig = plt.figure(figsize=(25,20))
tree = tree.plot_tree(dt3, 
                   feature_names=df1.columns[0:-1],  
                   class_names=df1.columns[1109],
                   filled=True)
#tree.plot_tree(dt3)

calculation(DecisionTreeClassifier(criterion='entropy', splitter= 'best', max_depth=3),X_train_scaled, y_train.values.ravel(),X_test_scaled,y_test)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y.values.ravel())
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

print(feat_importances.nlargest(10))

<h2>All Information 1 Second Model</h2>

This section repeats the same as above creating the same models but using only 1 second of the tracking data and the additional play information

In [ ]:
#loop to drop columns for 1 second model
columns=df1[df1.columns]
time=[15,20,25,30]
drop=[]
for i in columns:
    for t in time:
        if('t'+str(t) in i):
            drop.append(i)
            
#print(drop)

In [ ]:
dfone=df1.drop(columns=drop)

In [ ]:
columnsList = []
for col in dfone.columns:
    columnsList.append(col)
#print(columnsList)

In [ ]:
# try building model with only tracking data
# for first second instead of first 3 seconds
X1 = dfone[dfone.columns[0:-1]]
y1 = dfone[['result_Fair Catch']]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, 
                                                    test_size=0.20, 
                                                    random_state=0)

scaler = MinMaxScaler()
scaler.fit(X1_train)
X1_train_scaled = scaler.transform(X1_train)
X1_test_scaled = scaler.transform(X1_test)

**Logistic Regression**

Best Logistic Regression - 1 second model   
c=1  
Confusion matrix:   
  [[681  68]  
   [272  53]]  

Accuracy: 0.6834  
Precision: 0.438  
Recall: 0.1631  
AUC: 0.5361   

In [ ]:
# Default c = 1
for c in [1,2,3,4,5,6,7,8,9,10]: 
    calculation(LogisticRegression(C=c,max_iter=10000),X1_train_scaled,y1_train.values.ravel(),X1_test_scaled,y1_test)

**Random Forest Clasifier**

Best for Random Forest - 1 second  
 Size:  50  
Confusion matrix:   
    [[706  43]  
     [289  36]]  

Accuracy: 0.6909  
Precision: 0.4557  
Recall: 0.1108  
AUC: 0.5267  

In [ ]:
for size in range(50,160,10): 
    print("Size: ", size)
    calculation(RandomForestClassifier(n_estimators=size,random_state = 1),X1_train_scaled,y1_train.values.ravel(),X1_test_scaled,y1_test)

**Decision Tree**

Best DT - 1 second
Criterion:  gini  
Splitter:  best  
Confusion matrix:   
[[526 223]  
 [211 114]]  

Accuracy: 0.5959  
Precision: 0.3383  
Recall: 0.3508  
AUC: 0.5265  

with max_depth=3  
Confusion matrix:   
[[654  95]  
 [236  89]]  

Accuracy: 0.6918  
Precision: 0.4837  
Recall: 0.2738  
AUC: 0.5735  

In [ ]:
#gini, best 
for criterion in ["gini","entropy"]:
    for splitter in ["best","random"]:
        print("Criterion: ", criterion)
        print("Splitter: ", splitter)
        calculation(DecisionTreeClassifier(criterion=criterion, splitter=splitter),X1_train_scaled, y1_train.values.ravel(),X1_test_scaled,y1_test)

In [ ]:
from sklearn import tree
dt1 = DecisionTreeClassifier(criterion='gini', splitter= 'best', max_depth=3)
dt1= dt1.fit(X1_train, y1_train.values.ravel())
#y_pred = model_name.predict(X_test)
#visualizing gini-best tree
fig = plt.figure(figsize=(25,20))
tree = tree.plot_tree(dt1, 
                   feature_names=dfone.columns[0:-1],
                   class_names=dfone.columns[493],
                   filled=True)
#tree.plot_tree(dt1)
calculation(DecisionTreeClassifier(criterion='gini', splitter= 'best', max_depth=3),X1_train_scaled, y1_train.values.ravel(),X1_test_scaled,y1_test)


**K Nearest Neighbors**  
Best KNN- 1 second model   
Distance metric:  Euclidean  
k:  10  
Confusion matrix:   
[[686  63]  
 [280  45]]  

Accuracy: 0.6806  
Precision: 0.4167   
Recall: 0.1385  
AUC: 0.5272  

In [ ]:
# Default is p = 2, k = 5
for p in [1, 2]: 
    for k in [1,2,3,4,5,6,7,8,9,10]: 
        distance = {1: "Manhattan", 2: "Euclidean"}
        print("Distance metric: ", distance[p])
        print("k: ", k)
        calculation(KNeighborsClassifier(p=p,n_neighbors=k),X1_train_scaled,y1_train.values.ravel(),X1_test_scaled,y1_test)

<h2>3 Second Tracking Data Model</h2>

This section repeats the same process but instead of using the extra play information such as quarter and yardline number the only information captured is the tracking data for all 3 seconds.

In [ ]:
#deleting everything except positonaltracking data 
drop=[]
columns=df1.columns
for i in columns:
    if(i!='P1_t0_x'):
        drop.append(i)
    else:
         break
#print(drop)
dfpos= df1.drop(columns= drop)

In [ ]:
Xp = dfpos[dfpos.columns[0:-1]]
yp = dfpos[['result_Fair Catch']]
Xp_train, Xp_test, yp_train, yp_test = train_test_split(Xp, yp, 
                                                    test_size=0.20, 
                                                    random_state=0)

scaler = MinMaxScaler()
scaler.fit(Xp_train)
Xp_train_scaled = scaler.transform(Xp_train)
Xp_test_scaled = scaler.transform(Xp_test)

**Logistic Regression**

Best Logistic Regression - 3 second only tracking
Confusion matrix:   
[[657  92]  
 [266  59]]  
  
Accuracy: 0.6667  
Precision: 0.3907  
Recall: 0.1815  
AUC: 0.5294  

In [ ]:
# Default c = 1 
for c in [1,2,3,4,5,6,7,8,9,10]: 
    calculation(LogisticRegression(C=c,max_iter=10000),Xp_train_scaled,yp_train.values.ravel(),Xp_test_scaled,yp_test)

**Random Forest Classifier**

Best RFC - 3 second tracking  
Size:  140  
Confusion matrix:   
[[717  32]  
 [305  20]]  

Accuracy: 0.6862   
Precision: 0.3846  
Recall: 0.0615  
AUC: 0.5094  

In [ ]:
for size in range(50,160,10): 
    print("Size: ", size)
    calculation(RandomForestClassifier(n_estimators=size,random_state = 1),Xp_train_scaled,yp_train.values.ravel(),Xp_test_scaled,yp_test)

**Decision Tree Clasifier**  
Best Decision Tree - 3 second positional data model    
Criterion:  gini  
Splitter:  random  
Confusion matrix:   
[[518 231]  
 [199 126]]  

Accuracy: 0.5996  
Precision: 0.3529  
Recall: 0.3877  
AUC: 0.5396  

In [ ]:
from sklearn import tree
#gini random is the best one from down below so thats why I plot it 
dtp = DecisionTreeClassifier(criterion='gini', splitter= 'random')
dtp= dt1.fit(Xp_train_scaled, yp_train.values.ravel())
#y_pred = model_name.predict(X_test)
#visualizing gini-best tree
fig = plt.figure(figsize=(25,20))
tree = tree.plot_tree(dtp, 
                   feature_names=dfpos.columns[0:-1],
                   class_names=dfpos.columns[1078],
                   filled=True)
#tree.plot_tree(dtp)
calculation(DecisionTreeClassifier(criterion='gini', splitter= 'random'),Xp_train_scaled, yp_train.values.ravel(),Xp_test_scaled,yp_test)


In [ ]:
for criterion in ["gini","entropy"]:
    for splitter in ["best","random"]:
        print("Criterion: ", criterion)
        print("Splitter: ", splitter)
        calculation(DecisionTreeClassifier(criterion=criterion, splitter=splitter),Xp_train_scaled, yp_train.values.ravel(),Xp_test_scaled,yp_test)

**K Nearest Neighbor**  
best KNN - 3 second positional data model     
Distance metric:  Manhattan  
k:  10  
Confusion matrix:   
[[691  58]  
 [279  46]]  

Accuracy: 0.6862  
Precision: 0.4423  
Recall: 0.1415  
AUC: 0.5321  


In [ ]:
for p in [1, 2]: 
    for k in [1,2,3,4,5,6,7,8,9,10]: 
        distance = {1: "Manhattan", 2: "Euclidean"}
        print("Distance metric: ", distance[p])
        print("k: ", k)
        calculation(KNeighborsClassifier(p=p,n_neighbors=k),Xp_train_scaled,yp_train.values.ravel(),Xp_test_scaled,yp_test)

<h2> 1 Second tracking data model </h2>

This section repeats the last section process but with 1 second of trackiing data instead of 3.

In [ ]:
#positional data 1 second model 
columns=dfpos[dfpos.columns]
time=[15,20,25,30]
drop=[]
for i in columns:
    for t in time:
        if('t'+str(t) in i):
            drop.append(i)
dfpos1=dfpos.drop(columns=drop)


In [ ]:
Xp1 = dfpos1[dfpos1.columns[0:-1]]
yp1 = dfpos1[['result_Fair Catch']]
Xp1_train, Xp1_test, yp1_train, yp1_test = train_test_split(Xp1, yp1, 
                                                    test_size=0.20, 
                                                    random_state=0)

scaler = MinMaxScaler()
scaler.fit(Xp1_train)
Xp1_train_scaled = scaler.transform(Xp1_train)
Xp1_test_scaled = scaler.transform(Xp1_test)

**Logistic Regression**

Best Logistic Regression- 1 second tracking
Confusion matrix:   
[[717  32]  
 [318   7]]  

Accuracy: 0.6741  
Precision: 0.1795  
Recall: 0.0215  
AUC: 0.4894  

In [ ]:
# Default c = 1 
for c in [1,2,3,4,5,6,7,8,9,10]: 
    calculation(LogisticRegression(C=c,max_iter=10000),Xp1_train_scaled,yp1_train.values.ravel(),Xp1_test_scaled,yp1_test)

**Random Forest** 

Best RFC - 1 second tracking
Size:  150  
Confusion matrix:   
[[715  34]  
 [303  22]]  

Accuracy: 0.6862  
Precision: 0.3929  
Recall: 0.0677  
AUC: 0.5111  

In [ ]:
for size in range(50,160,10): 
    print("Size: ", size)
    calculation(RandomForestClassifier(n_estimators=size,random_state = 1),Xp1_train_scaled,yp1_train.values.ravel(),Xp1_test_scaled,yp1_test)

**Decision Tree**

Best Decision Tree - 1 second tracking

In [ ]:
for criterion in ["gini","entropy"]:
    for splitter in ["best","random"]:
        print("Criterion: ", criterion)
        print("Splitter: ", splitter)
        calculation(DecisionTreeClassifier(criterion=criterion, splitter=splitter),Xp1_train_scaled, yp1_train.values.ravel(),Xp1_test_scaled,yp1_test)

**KNN**

Best KNN- 1 second tracking

Distance metric:  Euclidean  
k:  10  
Confusion matrix:   
[[690  59]  
 [281  44]]  

Accuracy: 0.6834  
Precision: 0.4272  
Recall: 0.1354  
AUC: 0.5283  

In [ ]:
for p in [1, 2]: 
    for k in [1,2,3,4,5,6,7,8,9,10]: 
        distance = {1: "Manhattan", 2: "Euclidean"}
        print("Distance metric: ", distance[p])
        print("k: ", k)
        calculation(KNeighborsClassifier(p=p,n_neighbors=k),Xp1_train_scaled,yp1_train.values.ravel(),Xp1_test_scaled,yp1_test)

**SVM**

In [ ]:
for c in range(1, 21,2):
    c = c / 10
    print("C: ", c)
    calculation(SVC(C = c),Xp1_train_scaled,yp1_train.values.ravel(),Xp1_test_scaled,yp1_test)

**Conclusion**

Despite building several models with different levels of information in order to combat the curse of dimensionality the results of the accuracy were always aroud .65-.70 and the AUC score around .5 regardless of the model classifier and information given. the percision score was always a bit low as well. There was an issue with imbalanced data that we fix and then run all models the same in a different notebook. 